# Выявление особенностей словоупотребления в текстах Л. Н. Толстого методами дистрибутивной семантики

## Данные

На основе 90-томного собрания сочинений

In [1]:
import os
import re
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup
from statistics import mean
from pymystem3 import Mystem

In [7]:
tei_ns = {'tei': 'http://www.tei-c.org/ns/1.0'}
mystem = Mystem()

In [4]:
with open ('.\\data\\Final\\Альберт 5.xml','r', encoding='utf-8') as f:
    text = f.read()

In [75]:
text[:400]

'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:xi="http://www.w3.org/2001/XInclude">\n <teiHeader>\n  <fileDesc>\n   <titleStmt>\n    <title>\n     Альберт\n    </title>\n    <author>\n     Толстой Л.Н.\n    </author>\n    <respStmt>\n     <resp>\n подготовка TEI/XML\n </resp>\n     <name>\n      Мария Картышева, Евгений Можаев, Даниил Скоринкин, Елена Сидорова\n     </name>'

In [37]:
root = etree.parse('.\\data\\Final\\Альберт 5.xml').getroot()

In [77]:
for i in root.findall(".//tei:text/tei:div/tei:div/tei:p", tei_ns)[:3]:
    print (i.text)


     Пять человек богатых и молодых людей приехали в третьем часу ночи веселиться на петербургский балик.
    

     Шампанского было выпито много, большая часть господ были очень молоды, девицы были красивы, фортепьяно и скрипка неутомимо играли одну польку за другою, танцы и шум не переставали; но было как-то скучно, неловко, каждому казалось почему-то (как это часто случается), что всё это не то и ненужно.
    

     Несколько раз они усиливались поднять веселье, но притворное веселье было еще хуже скуки.
    


In [84]:
root.find(".//tei:biblScope", tei_ns).text

'\n       5\n      '

In [181]:
class TextItem(object):
    def __init__(self):
        self.title = None
        self.year = None
        self.vol = None
        self.text = ''
        self.type = None
        
    def __str__(self):
        s = "{}\n\n{}\n\n{}\n\n{}\n\n{}...".format(self.type, self.title, self.year, self.vol, self.text[:100])
        return s
    
    __repr__ = __str__

### Произведения

In [242]:
def get_one_writing(filename):
    TI = TextItem()
    root = etree.parse(filename).getroot()
    TI.type = 'composition'
    TI.title = root.find(".//tei:titleStmt/tei:title", tei_ns).text.strip()
    TI.year = root.find(".//tei:creation/tei:date", tei_ns).text.strip()
    TI.vol = root.find(".//tei:biblScope", tei_ns).text.strip()
    text = []
    for i in root.findall(".//tei:text/tei:div/tei:div/tei:p", tei_ns):   
        etree.strip_tags(i, 'tei:choice', 'tei:reg')
        #print(type(i))
        #i.remove(child)# .remove('tei:orig')
        text.append(str(i.text).strip())
    #print (text)
    TI.text = ' '.join(text)
    return TI

In [293]:
s = '<p rend="left"> <hi rend="italic"> <choice> <reg>Акциома</reg> <orig>Акцiома</orig> </choice>. </hi> Что не <choice><reg>имеет</reg><orig>имѣетъ</orig></choice> причины, то независимо. </p>'

In [332]:
root = etree.fromstring(s)
root

<Element p at 0x1bdafe74e88>

In [118]:
root = etree.parse('.\\data\\Final\\[«Ежели бы человек не желал…»] 1.xml')
root

In [77]:
reg = root.findall(".//tei:text/tei:div/tei:div/tei:p/tei:choice/tei:reg", tei_ns)
len(reg)

317

In [114]:
reg = root.findall(".//tei:text/tei:div/tei:div/tei:p/tei:choice/tei:reg", tei_ns)
len(reg)

317

In [119]:
choices = root.findall(".//tei:text/tei:div/tei:div/tei:p/tei:choice", tei_ns)
len(choices)
children = [i.getchildren() for i in choices]
#for i in choices:
#    print(i.getchildren())

In [120]:
for key, i in enumerate(choices):
    i.remove(children[key][1])

In [121]:
choices[0].getchildren()

[<Element {http://www.tei-c.org/ns/1.0}reg at 0x1b103de0848>]

In [124]:
for key, i in enumerate(choices):
    etree.strip_tags(i, 'tei:reg')

In [ ]:
for 

In [126]:
choices[0].getchildren()

[<Element {http://www.tei-c.org/ns/1.0}reg at 0x1b103de0848>]

In [137]:
etree.strip_tags(root, '{http://www.tei-c.org/ns/1.0}reg')

In [139]:
etree.strip_tags(root, '{http://www.tei-c.org/ns/1.0}choice')

In [167]:
etree.strip_tags(root, '{http://www.tei-c.org/ns/1.0}p')

In [168]:
reg = root.findall(".//tei:text", tei_ns)[0]

In [170]:
reg.text

'\n  \n   \n    '

In [169]:
etree.tostring(reg, encoding='cp1251')

b'<?xml version=\'1.0\' encoding=\'cp1251\'?>\n<text xmlns="http://www.tei-c.org/ns/1.0" xmlns:xi="http://www.w3.org/2001/XInclude">\n  \n   \n    <head xml:id="h49">\n *IX.\n <lb/>\n [\xce\xd2\xd0\xdb\xc2\xce\xca \xc1\xc5\xc7 \xc7\xc0\xc3\xcb\xc0\xc2\xc8\xdf.]\n </head>\n    \n 1) \xc5\xe6\xe5\xeb\xe8 \xe1\xfb \xf7\xe5\xeb\xee\xe2\xe5\xea \xed\xe5 \xe6\xe5\xeb\xe0\xeb, \xf2\xee \xe8 \xed\xe5 \xe1\xfb\xeb\xee \xe1\xfb \xf7\xe5\xeb\xee\xe2\xe5\xea\xe0. \xcf\xf0\xe8\xf7\xe8\xed\xe0 \xe2\xf1\xff\xea\xee\xe9 \xe4\xe5\xff\xf2\xe5\xeb\xfc\xed\xee\xf1\xf2\xe8 \xe5\xf1\xf2\xfc \xe6\xe5\xeb\xe0\xed\xe8\xe5. \xc6\xe5\xeb\xe0\xed\xe8\xe5 \xe5\xf1\xf2\xfc \xf1\xe8\xeb\xe0 \xe4\xe5\xff\xf2\xe5\xeb\xfc\xed\xee\xf1\xf2\xe8, \xed\xee \xf2\xe0\xea \xea\xe0\xea \xef\xf0\xe8\xf7\xe8\xed\xe0 \xe4\xe5\xff\xf2\xe5\xeb\xfc\xed\xee\xf1\xf2\xe8 \xe1\xfb\xe2\xe0\xe5\xf2 \xe4\xe2\xee\xff\xea\xe0, \xe8\xec\xff\xed\xed\xee \xe8\xeb\xe8 \xef\xf0\xee\xe8\xf1\xf5\xee\xe4\xe8\xf2\xfc \xee\xf2 \xe6\xe5\xeb\xe0\xed\xe8\

In [154]:
etree.strip_tags(root, '{http://www.tei-c.org/ns/1.0}div')

In [162]:
for p in root.findall(".//tei:text/tei:div/tei:div/tei:p", tei_ns):
    print (p.getchildren())

In [161]:
for p in root.findall(".//tei:text/tei:div/tei:div/tei:p", tei_ns):
    print (p.text)

In [50]:
for p in root.findall(".//tei:text/tei:div/tei:div/tei:p", tei_ns):
    for i in p:
        i.remove(i.find('orig'))
    #etree.strip_tags(p, 'hi')
    #for i in p.findall('choice'):
    #    i.remove(i.find('orig'))
    #    etree.strip_tags(i, 'reg')
    #print (p.text)

TypeError: Argument 'element' has incorrect type (expected lxml.etree._Element, got NoneType)

In [20]:
root.findall(".//tei:text/tei:div/tei:div/tei:p", tei_ns)[0].text

'\n 1) Ежели бы '

In [12]:
etree.strip_tags(root.findall(".//tei:text/tei:div/tei:div/tei:p", tei_ns)[0], 'hi')

In [3]:
etree.strip_tags(root, 'hi')

In [334]:
for i in root.findall('choice'):
    i.remove(i.find('orig'))
    etree.strip_tags(i, 'reg')

In [335]:
etree.strip_tags(root, 'choice')
root.text

'   Акциома .  Что не имеет причины, то независимо. '

In [331]:
for child in root:
    print(child.tag, child.attrib)

reg {}
reg {}


In [311]:
w = root.findall('hi')[0].findall('choice')
w

[<Element choice at 0x1bdafdb96c8>]

In [289]:
for profile in child.findall(".//profile[@updated='true']"):
    child.remove(profile)

In [281]:
root.text

' '

In [251]:
root.text

' '

In [243]:
get_one_writing('.\\data\\Final\\[«Ежели бы человек не желал…»] 1.xml').text

'1) Ежели бы Я желаю знать правду, желаю знать, что [такое] я, почему я ощущаю     2) Но,  Теперь  Все то, что  Теперь Воля не ограниченна, сама Теперь Теперь  '

In [ ]:
get_one_writing('.\\data\\Final\\Альберт 5.xml')

In [189]:
path = '.\\data\\Final\\'

In [191]:
for root, dirs, files in os.walk (path):
        for fl in files :
            print (fl)

Abrégé de l’Evangile 24.xml
AMOUR DE LA PATRIE 1.xml
Carthago delenda est 39.xml
Carthago delenda est. Черновое 39.xml
Letter to a Hindoo (Письмо к индусу) 37.xml
NOTES SUR LE SECOND CHAPITRE DES «CARACTÈRES» DE LA BRUYÈRE 1.xml
The hostelry (Гостиница) 37.xml
[Carthago delenda est] («Жизнь, та форма жизни»...) 27.xml
[LE PRÉSENT, LE PASSÉ ET LE FUTUR] 1.xml
[«Давно позабыл я о счастьи ...»] 1.xml
[«Дядинька Жданов и кавалер Чернов»] 3.xml
[«Ежели бы человек не желал…»] 1.xml
[«Ей, Марьяна, брось работу!..»] 1.xml
[«Знание музыки - субъективное и объективное...»] 1.xml
[«Начала» романа «Сто лет»] 17.xml
[«Начала» романа времен Петра I. Азовские походы] 17.xml
[«Начала» романа времен Петра I. Кожуховский поход] 17.xml
[«Начала» романа времен Петра I. Переход власти от Софьи к Петру] 17.xml
[«Начала» романа времен Петра I. Разгар Великой Северной войны] 17.xml
[«Начала» романа времен Петра I. Семья Алексея Михайловича и происхождение Петра I] 17.xml
[«Он не мог ни уехать, ни оставаться..

### Письма

### Дневники

## Обработка текста

### Теги

In [1]:
import nltk

In [19]:
a = 'Весной 1820 года Пушкина вызвали к военному генерал-губернатору Петербурга графу М. А. Милорадовичу для объяснения по поводу содержания его стихотворений (в том числе эпиграмм на Аракчеева, архимандрита Фотия и самого Александра I), несовместимых со статусом государственного чиновника. Шла речь о его высылке в Сибирь или заточении в Соловецкий монастырь. Лишь благодаря хлопотам друзей, прежде всего Карамзина, удалось добиться смягчения наказания. Его перевели из столицы на юг в кишинёвскую канцелярию И. Н. Инзова[34]. По пути к новому месту службы Александр Сергеевич заболевает воспалением лёгких, искупавшись в Днепре. Для поправления здоровья Раевские вывозят в конце мая 1820 года больного поэта с собой на Кавказ и в Крым[35]. По дороге семья Раевских и А. С. Пушкин останавливаются в г. Таганроге, в бывшем доме градоначальника П. А. Папкова (ул. Греческая, 40).'

In [21]:
a = 'Он находится на ул. Пушкина дом 17'

In [150]:
nltk.tokenize.sent_tokenize(tobetokenized, language='czech')

['Тогда Rambler Media (контролируется "Проф-медиа" Владимира Потанина) объявил о продаже Google 100% системы контекстной рекламы ЗАО "Бегун" за $140 млн (см.',
 'Ъ от 19 июля 2008 года).',
 'Всю жизнь доказывала, что она – лучшая, ноги стирая в кровь.',
 'За 22 года турне Павлова проехала по миру более полумиллиона километров, дала около 9 тыс.',
 'спектаклей.',
 'Так, например, «умышленное причинение смерти другому человеку», конечно же, – преступление, убийство (ст.',
 '105 УК РФ), но и… подвиг на войне.']

In [140]:
language_punkt_vars = nltk.tokenize.punkt.PunktLanguageVars
language_punkt_vars.sent_end_chars=('.','?','!','»','"','…')

In [83]:
print(nltk.tokenize.punkt.PunktLanguageVars.sent_end_chars)

('.', '?', '!', '»', '"', '…')


In [147]:
with open('.\\data\\traindata.txt', 'r', encoding='utf-8') as f:
    traindata = f.read()
trainer = nltk.tokenize.punkt.PunktTrainer(traindata, language_punkt_vars)
trainer.INCLUDE_ALL_COLLOCS = True 
trainer.INCLUDE_ABBREV_COLLOCS = True
params = trainer.get_params()
sbd = nltk.tokenize.punkt.PunktSentenceTokenizer(params)

  Abbreviation: [1.5195] ул
  Abbreviation: [0.5590] стр
  Abbreviation: [1.1180] руб
  Abbreviation: [4.1305] г
  Abbreviation: [1.5195] ст
  Abbreviation: [4.1305] п
  Abbreviation: [6.0781] т.п
  Abbreviation: [4.1305] н
  Abbreviation: [19.7537] см
  Abbreviation: [4.1305] м
  Abbreviation: [5.5900] тыс
0.00043010752688172043 0.005952380952380952 0.0 4650 336 2 2
0.00043010752688172043 0.005952380952380952 0.0 4650 336 2 2
0.0034408602150537634 0.005952380952380952 0.003245248029670839 4650 336 16 2
0.01032258064516129 0.02976190476190476 0.00880853036624942 4650 336 48 10
0.0034408602150537634 0.005952380952380952 0.003245248029670839 4650 336 16 2
0.0012903225806451613 0.005952380952380952 0.0009272137227630969 4650 336 6 2
0.003870967741935484 0.005952380952380952 0.0037088548910523874 4650 336 18 2
0.0008602150537634409 0.005952380952380952 0.0004636068613815484 4650 336 4 2
0.0017204301075268817 0.017857142857142856 0.0004636068613815484 4650 336 8 6
0.003010752688172043 0.011

In [149]:
with open('.\\data\\testdata.txt', 'r', encoding='utf-8') as f:
    tobetokenized = f.read()
for sentence in sbd.sentences_from_text(tobetokenized, realign_boundaries=True):
    print (sentence + '\n')

Тогда Rambler Media (контролируется "Проф-медиа" Владимира Потанина) объявил о продаже Google 100% системы контекстной рекламы ЗАО "Бегун" за $140 млн (см. Ъ от 19 июля 2008 года).

Всю жизнь доказывала, что она – лучшая, ноги стирая в кровь.

За 22 года турне Павлова проехала по миру более полумиллиона километров, дала около 9 тыс. спектаклей.

Так, например, «умышленное причинение смерти другому человеку», конечно же, – преступление, убийство (ст. 105 УК РФ), но и… подвиг на войне.



In [151]:
nltk.tokenize.sent_tokenize(tobetokenized, language='czech')

['Тогда Rambler Media (контролируется "Проф-медиа" Владимира Потанина) объявил о продаже Google 100% системы контекстной рекламы ЗАО "Бегун" за $140 млн (см.',
 'Ъ от 19 июля 2008 года).',
 'Всю жизнь доказывала, что она – лучшая, ноги стирая в кровь.',
 'За 22 года турне Павлова проехала по миру более полумиллиона километров, дала около 9 тыс.',
 'спектаклей.',
 'Так, например, «умышленное причинение смерти другому человеку», конечно же, – преступление, убийство (ст.',
 '105 УК РФ), но и… подвиг на войне.']